In [1]:
import numpy as np
import sys
frmo dwave.samplers import PlanarGraphSolver

# Procedure #1 - The Algorithm

In [35]:
def create_qubo_matrix(ttMatrix, demCostMatrix):
    V = len(ttMatrix[0])
    numBinVars = int((V * (V-1))/2) # simplified combination formula with r=2    
    
    Q = [ [0] * (numBinVars * numBinVars) for i in range(numBinVars * numBinVars)]  
    
    index = 0
    for v in range(V-1):
        for w in range(V-1-v):
            tempDemand = demCostMatrix[v][v+1+w]
            for n in range(V-1):
                for m in range(V-n-1):
                    tempCost = ttMatrix[n][n+1+m] * tempDemand
                    for i in range(V*V):
                        if (index == i):
                            Q[index][i] -= tempCost
                        else:
                            Q[index][i] += tempCost
                            Q[i][index] += tempCost
                    index += 1
    
    return Q

In [3]:
def utrp_solver(ttMatrix, demCostMatrix):
    
    Q = create_qubo_matrix(ttMatrix, demCostMatrix)
    
    return None

# Procedure #2: Gathering benchmark data to compare with other UTRP algorithms

## Methods for performance parameters

### Floyd-Warshall algorithm for shortest travel distance

In [4]:
# creates shortest path matrix for every pair of vertices
def floyd_warshall(ttMatrix):
    V = len(ttMatrix[0]) # number of vertices 
    minDist = [ [None] * V for i in range(V)] # creates a V*V array 
    
    for i in range(V):
        for j in range(V):
            minDist[i][j] = ttMatrix[i][j]
        
    for k in range(V):
        for i in range(V):
            for j in range(V):
                if minDist[i][j] > minDist[i][k] + minDist[k][j]:
                    minDist[i][j] = minDist[i][k] + minDist[k][j]
                    
    return minDist

### Average Travel Time (ATT)

In [5]:
def att(rSet, ttMatrix, demCostMatrix):
    V = len(ttMatrix[0])
    transitGraph = [ [sys.maxsize] * V for i in range(V)]
    
    # creates transit graph w/ travel times
    for i in range(V):
        transitGraph[i][i] = 0
    
    for r in rSet:
        for j in range(len(r)-1):
            tempV1 = r[j]
            tempV2 = r[j+1]
            transitGraph[tempV1][tempV2] = ttMatrix[tempV1][tempV2]      
            transitGraph[tempV2][tempV1] = ttMatrix[tempV2][tempV1]
            
    # finds min travel time for all pair of vertices  
    minDist = floyd_warshall(transitGraph)
    
    # finds average travel time - only uses upper triangular matrix
    ttt = 0
    totDemand = 0
    for i in range(V):
        for j in range(V-i):
            ttt += (demCostMatrix[i][j+i] * minDist[i][j+i]) 
            totDemand += demCostMatrix[i][j+i]
            
    return ttt/totDemand

### D0 - demand satisfied w/o transfers

In [6]:
def d0(rSet, demCostMatrix):
    V = len(ttMatrix[0]) 
    
    # creates copy of cost matrix
    copyCostMatrix = [ [0] * V for i in range(V)] # creates a V*V array 
    for i in range(V):
        for j in range(V):
            copyCostMatrix[i][j] = demCostMatrix[i][j]
    
    # calculates total demand
    totDemand = 0
    for i in range(V):
        for j in range(V-i):
            totDemand += copyCostMatrix[i][j+i]
    
    # calculates d0 as demand nums
    totD0 = 0
    for r in rSet:
        for i in range(len(r)):
            for j in range(len(r)-i-1):
                edge = [r[i], r[i+1+j]]
                totD0 += copyCostMatrix[edge[0]][edge[1]]
                # makes sure duplicate edges don't get added twice
                copyCostMatrix[edge[0]][edge[1]] = 0 
                copyCostMatrix[edge[1]][edge[0]] = 0 
                
    # returns d0 as a percentage   
    return totD0/totDemand 

### D1 - demand satisfied w/ 1 transfer

### D2 - demand satisfied w/ 2 transfers

### Dun - demand not satisfied

## Import and format data for travel time and travel demand for Mandl's Swiss Network

In [38]:
testRSet = [[13, 9, 10, 11, 3, 1, 2, 5], 
            [0, 1, 2, 5, 7, 14, 6, 9], 
            [8, 14, 5, 7, 9, 13, 12, 10], 
            [0, 1, 4, 3, 11, 10, 9, 12]]

ttMatrix = []

with open("MandlTravelTimes.txt", 'r') as file:
    for line in file:
        elements = line.strip().split()
        for i in range(len(elements)):
            elem = elements[i]
            if (elem == "Inf"):
                elements[i] = sys.maxsize
            else:
                elements[i] = int(elem)
        
        if (len(elements) > 0):
            ttMatrix.append(elements)
    
demCostMatrix = []

with open("MandlDemand.txt", 'r') as file:
    for line in file:
        elements = line.strip().split()
        for i in range(len(elements)):
            elements[i] = int(elements[i])
        
        if (len(elements) > 0):
            demCostMatrix.append(elements)

#print(att(testRSet, ttMatrix, demCostMatrix)) # works!
#print(d0(testRSet, demCostMatrix)) # works !
#print(len(create_qubo_matrix(ttMatrix, demCostMatrix))) # maybe works

11025


## Run algorithm and get results for Procedure #2

# Procedure #3 – Testing my algorithm on bus routes in downtown Seattle

## OSMnx graph of downtown Seattle

## Create transit time matrix

## Create travel demand cost matrix

## Run algorithm and get results for Procedure #3